# Plot Functional Connectivity

This script plots the Power-based functional connectivity. The plot can take many forms such as a matrix, glass brain, or circular graph. This was used to check many functional connectivities simultaneously for artifacts and problems.

In [1]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline

from common.data import get_subject_id_from_path, get_fc_data, get_data
from common.paths import HEALTHY, ADHD
from common.plotting import plot_connections, plot_fc_matrix
from common.power_atlas import to_power_fc_matrix
from sklearn.preprocessing import StandardScaler

## Plot One Power FC

In [9]:
sample_subject_id = "NDARAP912JK3"
fcs = get_fc_data()

print("Number of functional connectivity:", len(fcs))
print("Number of features (connections):", fcs[sample_subject_id].shape)

Number of functional connectivity: 880
Number of features (connections): (34716,)


In [ ]:
plot_connections(fcs[sample_subject_id], -1, 1, "99.5%", True, True, False)

## Plot Multiple Power FC

In [11]:
# NOTE: MUST MODIFY get_data() TO RETURN SUBJECT IDS OTHERWISE THIS FAILS
X, Y, demographics, population, X_ids = get_data(label_path=ADHD)
X = StandardScaler().fit_transform(X)

print(f'X: {X.shape} | Y: {len(Y.keys())} | Population: {population} | IDs: {len(X_ids)}')

X: (373, 34716) | Y: 6 | Population: adhd


In [15]:
filter_list = ['NDARKH171DG8', 'NDARJG298YVA', 'NDARFY009MNL', 'NDARPZ720WKW', 
               'NDARVC821LP9', 'NDARVC956HHP', 'NDARHD507TEF']

for fc, fc_id in zip(X_t, X_ids_t):
    print(fc_id)
    if fc_id not in filter_list:
        continue
    plot_fc_matrix(to_power_fc_matrix(fc), -1, 1, title=fc_id))